<a href="https://colab.research.google.com/github/kAIspeaker/k.A.I_speaker/blob/main/k_A_I_speaker_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
torch.cuda.is_available() 

True

In [ ]:
# !pip install sentencepiece

!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

# !git clone https://github.com/SKTBrain/KoBERT.git
# !cd KoBERT
# !pip install -r requirements.txt

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
# !pip install transformers==3# 버트 못가져옴
!pip install pytorch==1.8.1

from kobert_tokenizer import KoBERTTokenizer

from transformers import BertModel

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ##GPU 사용 시
device = torch.device("cuda:0")
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd


In [ ]:
chatbot_data = pd.read_excel("/content/drive/MyDrive/food_labeling.xlsx")
len(chatbot_data)
chatbot_data.sample(n=11)

chatbot_data.loc[(chatbot_data['Status'] == "결제"), 'Status'] = 0  
chatbot_data.loc[(chatbot_data['Status'] == "메뉴판"), 'Status'] = 1  
chatbot_data.loc[(chatbot_data['Status'] == "변경"), 'Status'] = 2  
chatbot_data.loc[(chatbot_data['Status'] == "영수증"), 'Status'] = 3  
chatbot_data.loc[(chatbot_data['Status'] == "재고_확인"), 'Status'] = 4  
chatbot_data.loc[(chatbot_data['Status'] == "정보_가격"), 'Status'] = 5  
chatbot_data.loc[(chatbot_data['Status'] == "정보_메뉴"), 'Status'] = 6  
chatbot_data.loc[(chatbot_data['Status'] == "정보_제휴"), 'Status'] = 7  
chatbot_data.loc[(chatbot_data['Status'] == "주문"), 'Status'] = 8  
chatbot_data.loc[(chatbot_data['Status'] == "추천"), 'Status'] = 9  
chatbot_data.loc[(chatbot_data['Status'] == "포인트_적립"), 'Status'] = 10  

data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Status'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

print(len(data_list))
print(data_list[1])
print(data_list[-1])



In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(len(dataset_train))
print(len(dataset_test))
print(dataset_train[0])
print(dataset_test[0])


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer,vocab=vocab,max_seq_length=max_len, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
vocab=tokenizer.vocab_file
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1')
vocab = nlp.vocab.BERTVocab.from_sentencepiece(vocab, padding_token='[PAD]')

# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)#문제아
tok = tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

data_train[0]
data_test[1]

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)



In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=11,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
        # _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device) )
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)



In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        
        valid_length= valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        
        valid_length= valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("결제")
            elif np.argmax(logits) == 1:
                test_eval.append("메뉴판")
            elif np.argmax(logits) == 2:
                test_eval.append("변경")
            elif np.argmax(logits) == 3:
                test_eval.append("영수증")
            elif np.argmax(logits) == 4:
                test_eval.append("재고_확인")
            elif np.argmax(logits) == 5:
                test_eval.append("정보_가격")
            elif np.argmax(logits) == 6:
                test_eval.append("정보_메뉴")
            elif np.argmax(logits) == 7:
                test_eval.append("정보_제휴")
            elif np.argmax(logits) == 8:
                test_eval.append("주문")
            elif np.argmax(logits) == 9:
                test_eval.append("추천")
            elif np.argmax(logits) == 10:
                test_eval.append("포인트_적립")
            
        print(">> 입력하신 내용은 " + test_eval[0] + " 에 대한 명령입니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if len(sentence) == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 햄버거 하나 주세요


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용은 주문 에 대한 명령입니다.


하고싶은 말을 입력해주세요 : 


In [ ]:
torch.save(model, 'k_A_I_speaker.pt')
